# 逆誤差伝播法

## 計算グラフ
ノードとエッジで表現されたグラフのこと

### 計算グラフでとく
計算グラフを用いた問題解決の手法
* 1. 計算グラフを構築する
* 2. 計算グラフ上で計算を左から右へ進める

計算を左から右へ進めるステップのことを**順伝播**という

**メリット**
* 計算をステップに分けて、処理を局所的に分けることができる => 単純化が可能

## 連鎖律
### 計算グラフの逆伝播
目的の微分を求める際には、逆伝播すると良い
例)
X円のりんごを２個買って、消費税10%かかるとき

$2X \times 1.1 = 2.2X$  

微分の値は、2.2となる

逆伝播でやると)  

$ 1.1 \times 2 = 2.2$


### 連鎖律とは
連鎖律の原理は、以下の通り  
合成関数について  
$z = (x+y)^2$

以下のように分離する  
$z = t^2$  
$t = x + y$

この時、zのxについての微分の値は以下で求められる  
$$
\frac { \partial z}{ \partial x} = \frac { \partial z}{ \partial t}  \frac{ \partial t}{ \partial x} 
$$

## 逆伝播

### 各ノードの逆伝播
* 加算：上流の値をそのまま下流へ流すだけ  
* 乗算：順伝播の時の入力信号以外の変数をかけた値にする(p.154がわかりやすい)

## 単純なレイヤの実装
### 乗算レイヤの実装

In [3]:
# 乗算レイヤのクラス
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [4]:
# 順伝播の例
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price) # 220

220.00000000000003


In [5]:
# 逆伝播の例
dpirce = 1
dapple_price, dtax = mul_tax_layer.backward(dpirce)

dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


### 加算レイヤの実装 

In [6]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

In [7]:
# 加算＋乗算の例
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_prce = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_prce, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dpirce)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange_num, dorange, dtax)

715.0000000000001
110.00000000000001 2.2 165.0 3.3000000000000003 650


## 活性化レイヤの実装
### ReLUレイヤ

$$
\begin{eqnarray}
\frac { \partial y}{ \partial x} =\left\{ \begin{array}{ll}
1 & (x > 0  ) \\
0 & (x  \leqq 0  ) \\
\end{array} \right.
\end{eqnarray}
$$

In [8]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx


In [13]:
# テスト
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])

# layer
relu_layer = Relu()
test = relu_layer.forward(x)
print(test)

[[ 1.  0.]
 [ 0.  3.]]


### シグモイドレイヤ
$$
\frac { \partial y}{ \partial x} = \frac {exp(-x)} {(1 + exp(-x))^2} = y(1-y)
$$

In [17]:
# 実装
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(slef, x):
        out = 1 / (1 + np.exp(-x))
        slef.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

## Affine / Softmax レイヤの実装
### Affineレイヤ
行列の積の計算を行うレイヤー
* 逆伝播の計算は、注目している変数でない変数の転置行列をかける(p.170)
 
$$ 
\frac { \partial L}{ \partial X} = \frac { \partial L}{ \partial Y} \cdot {W}^{\mathrm{T}}
$$

$$
\frac { \partial L}{ \partial W} = {X}^{\mathrm{T}}  \cdot \frac { \partial L}{ \partial Y}
$$

### バッチ版Affineレイヤ

In [16]:
# 実装(サンプル・・・データが２個の場合)
class Affine:
    def __init__(self, W, b):
        slef.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x 
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

### ソフトマックス With Loss レイヤについて
以下の組み合わせで設計すると、逆伝播が綺麗に伝わる
* 恒等関数＋２乗和誤差
* シグモイド関数+交差エントロピー誤差

In [21]:
# 実装

def softmax(x):
    c = np.max(x)
    exp_a = np.exp(x - c) # オーバーフロー対策
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

def cross_entropy_error(y, t):
    if y.ndim == 1:                       # 全て２次元データにする
        t = t.reshape(1, t.size)   
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return - np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None   # 損失
        self.y = None       # softmaxの出力
        self.t = None       # 教師データ
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size  # 割るのに注意！データ１個あたりに変換
        
        return dx

## 誤差逆伝播法の実装

In [7]:
# 実装

import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * \
                                      np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                                      np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = \
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        print(y)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        
        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        
        for layer in layers:
            dout = layer.backward(dout)
            
        # 設定
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW 
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

In [8]:
import sys, os 
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
# from common.two_layer_net import TwoLayerNet

# データの読み込み
(x_train, t_train), (x_test, t_test) = \
load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
x_batch = x_train[:3]
t_batch = t_train[:3]
grad_numerical = network.numerical_gradient(x_batch, t_batch) 
grad_backprop = network.gradient(x_batch, t_batch)

# 各重みの絶対誤差の平均を求める
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:4.1752570479053724e-10
b1:2.5120600439805084e-09
W2:6.5015458408767405e-09
b2:1.4039712413621564e-07
